In [1]:
from classes.Dataset import Dataset
from classes.Process_Stages import Process_Stages

from run_models.gensim.classes.Dataset_Gensim import Dataset_Gensim
from run_models.gensim.classes.Process_Stages_Gensim import Process_Stages_Gensim

neural_course = Dataset(
    df_name="neural_course", 
    model_name="gensim", 
    language="english", 
    process_stages = Process_Stages(
        basic_processed=True, 
    ),
)

neural_course.run_all()


File neural_course not found in (data_saved/basic_processed)
basic processing starting for neural_course


100%|██████████| 646/646 [11:19<00:00,  1.05s/it]

saving new basic_processed phase for: neural_course


In [2]:
vars(neural_course.process_stages)


{'basic_processed': False}